# Init

In [1]:
from decouple import Config, RepositoryEnv
from ollama_interaction import embeding_ollama_request,generate_ollama_request

import pandas as pd
import json

In [2]:
config = Config(RepositoryEnv('.env'))

ollama_api_url = config('OLLAMA_API_URL')
ollama_bearer_token = config('OLLAMA_BEARER_TOKEN')

In [32]:
# context option are based on https://github.com/NVIDIA/RULER
# If model is not on doc we take the nearest one

# best context is based on  Effective length
# max context is base on claimed length

with open('model-data.json','r') as file:
    model_data = json.load(file)

for item in list(model_data.keys()):
    print('model :',item)

model : mistral-nemo:latest
model : mistral-small:latest
model : qwen2.5:14b
model : qwen2.5:32b
model : llama3.2:1b
model : llama3.1:latest


# Scoring generation

In [33]:
data = pd.read_csv("hard-database/data_at_select_ai.csv",index_col='id')

data = data[(data['token_numb_description']<5000)&(data['token_numb_description']>500)]

In [70]:
def gen_prompt_sub_scoring(sub_description,project_description,max_score=5,min_score=-5):
    prompt_system = """Tu es une IA qui aide l'utilisateur à déterminer la compatibilité d'une aide ou subvention par rapport à la description de son projet.
    
    Tu ne dois pas être bavard et te conformer totalement et strictement aux demandes de l'utilisateur.
    
    Ne fournis aucune explication, aucun mot supplémentaire, ni aucun caractère additionnel dans ta réponse.
    """
    
    prompt_user = f"""**Instructions :**

    Attribue une note sous forme d'entier entre {min_score} et {max_score}, selon les critères suivants :

    {max_score} : L'aide ou la subvention correspond parfaitement aux objectifs et aux besoins du projet.
    Les objectifs de l'aide/subvention sont totalement alignés avec les besoins du projet.
    Les conditions et exigences de l'aide/subvention sont complètement satisfaites par le projet.

    {int(max_score*0.2)} : L'aide ou la subvention correspond très bien aux objectifs et aux besoins du projet.
    Les objectifs de l'aide/subvention sont majoritairement alignés avec les besoins du projet.
    La plupart des conditions et exigences de l'aide/subvention sont satisfaites par le projet.

    {0} : L'aide ou la subvention correspond partiellement aux objectifs et aux besoins du projet.
    Les objectifs de l'aide/subvention sont partiellement alignés avec les besoins du projet.
    Certaines conditions et exigences de l'aide/subvention sont satisfaites par le projet.

    {int(min_score*0.2)} : L'aide ou la subvention a une correspondance minimale avec les objectifs et les besoins du projet.
    Les objectifs de l'aide/subvention sont rarement alignés avec les besoins du projet.
    Peu de conditions et exigences de l'aide/subvention sont satisfaites par le projet.

    {min_score} : La description du projet est insuffisamment détaillée ou trop vague, ou l'aide ou la subvention n'a aucune correspondance avec les objectifs et les besoins du projet.
    Les objectifs de l'aide/subvention ne sont pas du tout alignés avec les besoins du projet.
    Aucune des conditions et exigences de l'aide/subvention n'est satisfaite par le projet.

    Analyse les informations suivantes pour déterminer la compatibilité entre l'aide ou la subvention et le projet de l'utilisateur.
    
    - **Aide ou subvention à analyser :**
    {sub_description}
    
    - **Projet de l'utilisateur :**
    {project_description}
    
    **Instructions :**

    - Utilise ces informations pour attribuer une note selon les critères précédemment définis. **Ne me renvoie que la note sans aucunes explications.**"""

    return prompt_system, prompt_user

In [37]:
with open("project-description-sample.json",'r') as file:
    project_descrpition_list = json.load(file)

In [97]:
seed_number = 5
max_retry = 10
row_list = []

model = "llama3.1:latest"
model_options = model_data[model]

score_sub_request_options = {
    "num_ctx": 16384,
    "num_predict": 2
}

error_list = []

request_options = score_sub_request_options

for project_descrpition_key in project_descrpition_list:
    project_description = project_descrpition_list[project_descrpition_key]
    print('-------------------------')
    print(project_description)
    for i, row in data.iterrows():
        print('---------')
        sub_description = row['description']
        sub_eligibility = row['eligibility']
        prompt_system,prompt_user = gen_prompt_sub_scoring(sub_description,project_description,max_score=5,min_score=-5)

        score_sub = 0
        seed = 0
        scoring_made = 0
        retry = 0
        while scoring_made < seed_number and retry < max_retry:
            seed += 1
            response = generate_ollama_request(
                prompt_system=prompt_system,
                response_format=None,
                prompt_user=prompt_user,
                ollama_api_url=ollama_api_url,
                bearer_token=ollama_bearer_token,
                model_options = model_options,  # Default to None
                request_options= request_options,  # Default to None
                seed=seed,
                )
            if response:
                try :
                    response_filtred = response['response'].replace(' ','').replace('\n','')
                    score_seed = int(response_filtred)
                    if score_seed > 5:
                        score_seed =5
                    if score_seed < -5:
                        score_seed = -5
                    score_sub+=score_seed
                    retry = 0
                    scoring_made+=1
                    print(score_sub)
                except Exception as error:
                    retry += 1
                    error_list.append(response_filtred)
                    print(f"error : {response['response']}")
        row['project'] = project_description
        row['project_score'] = score_sub
        row['scoing_made'] = scoring_made
        row_list.append(row)

-------------------------
Revitalisation d'une zone humide
---------
1
0
-1
-1
-2
---------
error : 0 (-
-1
-6
-5
-10
-10
---------
0
-1
-2
2
1
---------
1
0
-1
4
3
---------
0
-1
-2
-2
-3
---------
0
-1
-6
-5
-10
---------
1
0
-5
-1
-6
---------
1
0
-1
4
-1
---------
4
3
2
3
2
---------
1
-4
-9
-4
-9
---------
error : 0 (-
-1
-2
-1
-2
-1
---------
0
-5
-10
-10
-15
---------
0
-1
-2
2
1
---------
0
-1
-2
2
1
---------
1
0
-5
0
-5
---------
error : 0 (-
-1
-2
3
2
2
---------
1
0
-1
4
3
---------
4
3
-2
3
-2
---------
0
-1
-6
-5
-10
---------
1
0
-5
-1
-6
---------
1
0
-1
4
-1
---------
1
0
-5
0
-5
---------
4
3
-2
3
-2
---------
0
-1
-6
-1
-6
---------
3
2
-3
1
-4
---------
0
-1
-2
-1
-2
---------
1
0
-1
4
3
---------
0
-1
-2
3
2
---------
1
0
-1
0
-1
---------
4
3
2
7
2
---------
1
-4
-9
-4
-9
---------
5
4
3
7
6
---------
error : 0 (-
-1
-2
-1
-2
-1
---------
4
-1
-6
-1
-6
---------
1
-4
-9
-9
-14
---------
1
0
-5
-10
-15
---------
error : 0 (-
-1
-2
-1
-2
-2
---------
error : 0 (-
-5

# Question generation

In [98]:
data_score_sample = pd.DataFrame(row_list)

In [102]:
data_score_sample

,slug,url,name,name_initial,short_title,financers,financers_full,instructors,instructors_full,programs,...,is_live,description_md,eligibility_md,token_numb_description_md,token_numb_description,token_numb_eligibility_md,token_numb_eligibility,project,project_score,scoing_made
162955,appel-a-projets-pedagogiques-culture-cheval-an...,/aides/appel-a-projets-pedagogiques-culture-ch...,Développer des projets pédagogiques en lien av...,Appel à projets pédagogiques Culture Cheval – ...,NaN,['Conseil départemental de la Manche'],"[{'id': 164, 'name': 'Conseil départemental de...",[],[],[],...,True,**Nature de l'aide**\n\nLa Manche étant une te...,**Nature des projets éligibles**\n\nEnseignant...,501,610,380,444,Revitalisation d'une zone humide,-2,5
162956,appel-a-manifestation-dinteret-pour-loccupatio...,/aides/appel-a-manifestation-dinteret-pour-loc...,Candidater à l'appel à manifestation d’intérêt...,Appel à manifestation d’intérêt pour l’occupat...,NaN,"[""Conseil régional de Provence-Alpes-Côte d'Az...","[{'id': 93, 'name': ""Conseil régional de Prove...",[],[],[],...,True,"Aux termes d’une convention avec l’Etat, appro...",Tout projet soumettant un dossier complet comp...,1320,1595,1047,1306,Revitalisation d'une zone humide,-10,5
162957,passeurs-dimages-en-bourgogne-franche-comte,/aides/passeurs-dimages-en-bourgogne-franche-c...,Mener des projets d'éducation à l'image sur le...,Passeurs d'images,NaN,['Ministère de la Culture'],"[{'id': 96, 'name': 'Ministère de la Culture',...",[],[],[],...,True,**Qu'est\-ce que le dispositif Passeurs d’imag...,La coordination Passeurs d’images Bourgogne\-F...,548,1203,479,882,Revitalisation d'une zone humide,1,5
162965,ecoconception-textile-dhabillement-texhabi,/aides/ecoconception-textile-dhabillement-texh...,Soutenir les projets d’écoconception textile e...,Écoconception textile d'habillement - TEXHABI,NaN,['ADEME'],"[{'id': 22, 'name': 'ADEME', 'logo': 'https://...",[],[],[],...,True,Cet appel à projets a pour objet de **soutenir...,NaN,658,728,0,0,Revitalisation d'une zone humide,3,5
162966,aqacia-2024-gerer-les-pollutions-a-lozone-et-s...,/aides/aqacia-2024-gerer-les-pollutions-a-lozo...,Gérer les pollutions à l’ozone et sectorielles,AQACIA 2024 - Gérer les pollutions à l’ozone e...,NaN,['ADEME'],"[{'id': 22, 'name': 'ADEME', 'logo': 'https://...",[],[],[],...,True,**1\. Présentation générale du programme AQACI...,NaN,1187,1342,0,0,Revitalisation d'une zone humide,-3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117417,ed52-faire-appel-a-un-atelier-chantier-dinsert...,/aides/ed52-faire-appel-a-un-atelier-chantier-...,Faire appel à un Atelier et Chantier d’Inserti...,NaN,NaN,"['ID77', ""Agence départementale d'insertion - ...","[{'id': 2019, 'name': 'ID77', 'logo': 'https:/...",[],[],[],...,True,\n Favoriser l'insertion professionnelle grâce...,Collectivités de Seine\-et\-Marne,654,865,40,38,Le projet consiste en l’aménagement d’un terra...,6,5
119710,e96b-gerer-les-rivieres-et-les-millieux-humides,/aides/e96b-gerer-les-rivieres-et-les-millieux...,Préserver et restaurer les cours d'eau et les ...,Gestion des rivières et des milieux humides,NaN,"[""Conseil départemental de l'Eure""]","[{'id': 139, 'name': ""Conseil départemental de...",[],[],[],...,True,\n Le Conseil départemental propose aux collec...,\n**Conditions d'attribution** \n\n\n\n\n Les ...,1312,1667,1113,1341,Le projet consiste en l’aménagement d’un terra...,-9,5
121012,3903-aider-a-lamelioration-des-forets-communal...,/aides/3903-aider-a-lamelioration-des-forets-c...,Protéger la forêt,AIDE À L’AMÉLIORATION DES FORÊTS COMMUNALES E...,NaN,['Conseil départemental des Bouches-du-Rhône'],"[{'id': 126, 'name': 'Conseil départemental de...",[],[],[],...,True,\n**CONTENU DU PROGRAMME** \n\n\n\n\n**Sont su...,\n**BÉNÉFICIAIRES :** \n les communes et group...,1490,1869,89,103,Le projet consiste en l’aménagement d’un terra...,1,5
123492,69c0-encourager-et-accompagner-les-collectivit...,/aides/69c0

In [103]:
data_score_sample.to_csv("hard-database/data_project_scoring.csv")

In [95]:
data_score_sample_select = data_score_sample[data_score_sample['project_score']>7]

In [83]:
row = data_score_sample_select[['description','eligibility','project']].iloc[0]

sub_description,sub_eligibility,project_description = row['description'],row['eligibility'],row['project']

In [344]:
def gen_prompt_question(sub_description,sub_eligibility,project_description,num_question=3):
    prompt_system = """Tu es une IA qui aides l'utilisateur à se questionner sur la description de son projet.

    Tu ne dois pas être trop bavard et te conformer totalement et strictement aux demandes de l'utilisateur.

    Tu réponderas exclusivement au format JSON et sous la forme exacte que te demande l'utilisateur.
    """
    prompt_user = f"""**Instruction**
    
    Poses {num_question} questions en lien avec l'aide ou la subvention et son projet.
    N'hésite pas à utiliser des exemples pour illustrer.

    Evite les questions  trop compliquée, et cherche à obtenir plus de détails sur le projet, en lien avec la subvention proposée.

    Tu répondras au format JSON suivant : {{"Q1": str, "Q2": str, "Q3": str}}


    Analyse les informations suivantes pour déterminer les {}.
    
    - **Aide ou subvention à analyser :**
    {sub_description}
    
    - **Critères potentiels d'éligibilité :**
    {sub_eligibility}
    
    - **Projet de l'utilisateur :**
    {project_description}
    
    """

    return prompt_system, prompt_user

In [374]:
num_question = 3
seed = 0

score_sub_request_options = {
    "num_ctx": 16384,
    "num_predict": 1024
}

request_options = score_sub_request_options

prompt_user = gen_prompt_user_question(sub_description,sub_eligibility,project_description,num_question)

response = simple_ollama_request(
    prompt_system=prompt_system,
    response_format="json",
    prompt_user=prompt_user,
    ollama_api_endpoint=ollama_api_endpoint,
    bearer_token=ollama_bearer_token,
    model_options = model_options,  # Default to None
    request_options= request_options,  # Default to None
    seed=seed,
    )

In [357]:
len(response['context'])

1924

In [372]:
json_text = response['response']

# Remplacer les caractères HTML par une apostrophe
json_text = json_text.replace('&#39;', "'")

# Convertir la chaîne en dictionnaire Python
response_json = json.loads(json_text)


In [375]:
response_json

{'Q1': "Quel est le principal objectif de votre projets revitalisation d'une zone humide ?\n",
 'Q2': "Dans quelle région et dans quel secteur ou territoire se situez-vous pour impliquer l'aide à cette opération.\n",
 'Q3': "Pourriez vous décrire brièvement le projet de revitalization, son objectif principal?\nPuisque je ne suis pas en train d'aborder les critères du programme LEADER et que j'ai besoin plus des informations pour savoir si ma demande est viable. Vous pouvez donc me donner l’ensemble pertinent sur votre type opération ?\n"}